In [657]:
import pymysql as psql
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import datetime
import pymysql.cursors

In [658]:
class sqlServer(object):  # 定义sqlServer类，用来连接到mysql获取数据
    """docstring for sqlServer"""

    def __init__(self, server={
        'host': '121.40.211.161',
        'port': 58799,
        'user': 'hhh',
        'password': '123456',
        'db': 'UV',
        'charset': 'utf8mb4',
        'cursorclass': pymysql.cursors.DictCursor,
    }, table='`TOTAL_INFO_T`'):
        self.server = server
        self.table = table
        self.allData = self.sqlGetData(sql="SELECT * FROM "+self.table)

    def sqlGetData(self, sql):
        connection = pymysql.connect(**self.server)
        try:
            with connection.cursor() as cursor:
                # 执行sql语句，进行查询
                cursor.execute(sql)
                # 获取查询结果
                result = cursor.fetchall()
            # 没有设置默认自动提交，需要主动提交，以保存所执行的语句
            connection.commit()

        finally:
            connection.close()
        df = pd.DataFrame(result)
        return df

# 比较date1和date2的指定列
def getWowg(date1, date2, columns):
    allHosid = np.union1d(date1.index, date2.index)
    date1All = date1.reindex(allHosid, fill_value=0)
    date2All = date2.reindex(allHosid, fill_value=0)
    uvAllDay = pd.merge(date1All, date2All,
                        left_index=True, right_index=True)
    for column in columns:
        compareColumInput = [column+'_x', column+'_y']
        compareColumOutput = 'wowg_'+column
        wowg = (
            uvAllDay[compareColumInput[0]]-uvAllDay[compareColumInput[1]])/uvAllDay[compareColumInput[1]]
        uvAllDay[compareColumOutput] = wowg
    return uvAllDay

# 获取x天之前的日期
def getDate(x):
    today = datetime.date.today()
    oneday = datetime.timedelta(days=x)
    day = today-oneday
    return day

# 提取n天和n+r天前的数据
def getDateData(allData, apData ,n=1, r=7):
    d1 = n
    d2 = d1+r
    day1 = getDate(1)
    day2 = getDate(8)
    uvDay1 = allData[allData['log_date'] == day1].reset_index(
        drop=True).set_index('hos_id').drop('log_date', axis=1)
    uvDay2 = allData[allData['log_date'] == day2].reset_index(
        drop=True).set_index('hos_id').drop('log_date', axis=1)
    apDay1 = apData[apData['log_date'] == day1].set_index('hos_id').drop('log_date',axis = 1)
    apDay2 = apData[apData['log_date'] == day2].set_index('hos_id').drop('log_date',axis = 1)
    uvDay1 = pd.merge(uvDay1,apDay1,left_index = True,right_index = True,how = 'left')
    uvDay2 = pd.merge(uvDay2,apDay2,left_index = True,right_index = True,how = 'left')
    uvDay1.insert(0,'ap_count',uvDay1.pop('ap_count'))
    uvDay2.insert(0,'ap_count',uvDay2.pop('ap_count'))
    d2 = uvDay2[uvDay2['dhcp'] > 200]
    d1 = uvDay1.reindex(d2.index)
    return uvDay1,uvDay2,d1, d2

# 显示数据，column表示要检查的字段，limit表示阈值
def showData(data, column, limit, total, data400):
    uvAllDay = data[[column+'_x', column+'_y', 'wowg_'+column]]
    table = pd.merge(uvAllDay[
        uvAllDay['wowg_'+column] < limit], total, left_index=True, right_index=True, how='left')
    tableShow = pd.merge(table, data400, left_on='Hospital',
                         right_on=r'医院名称', how='left')
    return tableShow

# 获取r天内的400报修信息
def get400Data(path, r):
    day = getDate(r)
    data400 = pd.read_excel(path)
    data400.columns = data400.loc[0].values.tolist()
    data400 = data400.drop(0)
    data4001 = pd.DataFrame([])
    data4001[[r'报修日期', r'解决问题日期']] = data400[
        [r'报修日期', r'解决问题日期']].dropna().astype('datetime64[ns]')
    data4001[[r'医院名称', r'报修问题', r'解决方式', r'解决人']] = data400[
        [r'医院名称', r'报修问题', r'解决方式', r'解决人']]
    return data4001[(data4001[r'报修日期'] > day)]


# 获取uv原始数据和总表数据
def getRawData(server={
        'host': '121.40.211.161',
        'port': 58799,
        'user': 'hhh',
        'password': '123456',
        'db': 'hive_statistical_results',
        'charset': 'utf8mb4',
        'cursorclass': pymysql.cursors.DictCursor},
        table="`funnel_model2`"):
    uvServer = sqlServer(server, table)
    apData = uvServer.sqlGetData( "SELECT * FROM `ap_count`")
    uvData = uvServer.allData
    totalServer = sqlServer()
    totalInfo = totalServer.allData
    totalInfo1 = totalInfo[['Hospital', 'CurrentBroadbandBandwidth', 'BroadbandType',
                            'GatewayVendor', 'GatewayType', 'ApVendor', 'ApType', 'ApVolume', 'GWID', 'GWIP']]
    totalInfo1['hos_id'] = totalInfo['HosID'].drop(0).astype('int32')
    totalInfo1 = totalInfo1.set_index('hos_id')
    return uvData, totalInfo1,apData

# 清洗数据
def dataClean(uvData,apData):
    uvData = uvData.replace('\\N', 0).fillna(0).sort_values(
        by='log_date', ascending=False).reset_index(drop=True)
    uvDataGood = uvData[['hos_id','dhcp', 'portal', 'prelogin',
                         'login', 'webforward', 'hardforward']].astype('int32')
    uvDataGood['log_date'] = uvData['log_date'].astype('datetime64[ns]')
    uvDataGood['forward_rate'] = uvDataGood['hardforward']/uvDataGood['dhcp']
    uvDataLite = uvDataGood.drop(
        ['portal', 'prelogin', 'login', 'webforward'], axis=1)
    apDataGood = apData[['hos_id','ap_count']].astype('int64')
    apDataGood['log_date'] = apData['log_date'].astype('datetime64[ns]')
    return uvDataGood,apDataGood

In [659]:
def analysis(interestedColumns, uvDataGood,apDataGood,day=1, cycle=7, limit=-0.4):
    analysisList = {}
    columnName = {''}
    path = './' + r'400与回访医院报修记录单统计表 (更改版).xlsx'
    columns = interestedColumns
    uvDay1, uvDay2,uvDay1Lite, uvDay2Lite = getDateData(uvDataGood, apDataGood,n=day, r =cycle)
    data400 = get400Data(path, day+cycle)
    uvAllDay = getWowg(uvDay1Lite, uvDay2Lite, columns)
    for column in columns:
        outPut = showData(uvAllDay, column, limit, totalInfo, data400).set_index("Hospital")
        outPut[r'现象'] = str(column) + '<' + str(limit)
        analysisList[column] = outPut
    return analysisList

In [660]:
#依靠经验值来构建的决策树

def decisionTree(data,rawData,apLimit = -0.1,forwardLimit = -0.4 ,globalLimit = -0.4):
    index = data.index #获得目录
    splitTerm0 = data[index[0]] <= globalLimit #分裂条件0
    if len(index) > 1:
        splitTerm1 = data[index[1]] <= 0.5*globalLimit #分裂条件1
    
    #"""决策树第一层"""
    #如果data的第一行索引名为'ap_count',并且第一行值等于-1，那么返回‘网关离线’。（AP全掉了）
    if index[0] == 'ap_count' and data[index[0]] == -1:
        return '网关离线' 
    
    #如果data的第一行索引名为'ap_count',并且第一行值小于apLimi同时不等于-1，那么返回‘AP离线’。（AP掉了一部分）
    elif index[0] == 'ap_count' and data[index[0]] <= apLimit and data[index[0]] != -1:
        return 'AP离线'
    
    #如果data的第一行索引名为'ap_count',并且第一行值大于apLimit，那么data去除第一行，其他参数保持不变，进行递归
    #（如果AP数量没有下降，那么进行下一步判断）
    elif index[0] == 'ap_count' and data[index[0]] > apLimit:
        return decisionTree(data[1:],rawData,apLimit,forwardLimit,globalLimit)
    
    
    #"""决策树第二层"""
    #如果data的第一行索引名为'dhcp',并且同时满足分裂条件0和1，那么返回'未知'。（莫名其妙的DHCP数就下降了）
    elif index[0] == 'dhcp' and splitTerm0 and splitTerm1:
        return '未知  '+index[0]+'<'+str(globalLimit)
    
    #如果data的第一行索引名为'dhcp',并且满足分裂条件0,不满足分裂条件1，那么返回'DHCP统计错误'。（只有DHCP数下降了）
    elif index[0] == 'dhcp' and splitTerm0 and not splitTerm1:
        return 'DHCP统计错误'
    
    #如果data的第一行索引名为'dhcp',并且不满足分裂条件0，那么data去除第一行，其他参数保持不变，进行递归
    #（如果DHCP数没有下降，那么进行下一步判断）
    elif index[0] == 'dhcp' and not splitTerm0:
        return decisionTree(data[1:],rawData,apLimit,forwardLimit,globalLimit)
    
    
    #"""决策树第三层"""
    #如果data的第一行索引名为'portal',并且同时满足分裂条件0和1，那么返回'重定向故障'。（莫名其妙的portal数就下降了）
    elif index[0] == 'portal' and splitTerm0 and splitTerm1:
        return '重定向故障'
    
    #如果data的第一行索引名为'portal',并且满足分裂条件0不满足分裂条件1，那么返回'portal打点故障'。（只有portal数下降了）
    elif index[0] == 'portal' and splitTerm0 and not splitTerm1:
        return 'portal打点故障'
    
    #如果data的第一行索引名为'portal',并且不满足分裂条件0，那么data去除第一行，其他参数保持不变，进行递归
    #（如果portal数没有下降，那么进行下一步判断）
    elif index[0] == 'portal' and not splitTerm0:
        return decisionTree(data[1:],rawData,apLimit,forwardLimit,globalLimit)
    
    
    #"""决策树第四层"""
    #如果data的第一行索引名为'prelogin',并且同时满足分裂条件0和1，那么返回'带宽负载较高（疑似）'。（预登陆人数变少了，可能是带宽造成的）
    elif index[0] == 'prelogin' and splitTerm0 and splitTerm1:
        return '带宽负载较高（疑似）'
    
    #如果data的第一行索引名为'prelogin',并且满足分裂条件0不满足分裂条件1，那么返回'prelogin打点故障'。（只有prelogin数下降了）
    elif index[0] == 'prelogin' and splitTerm0 and not splitTerm1:
        return 'prelogin打点故障'
    
    #如果data的第一行索引名为'prelogin',并且不满足分裂条件0，那么data去除第一行，其他参数保持不变，进行递归
    #（如果prelogin数没有下降，那么进行下一步判断）
    elif index[0] == 'prelogin' and not splitTerm0:
        return decisionTree(data[1:],rawData,apLimit,forwardLimit,globalLimit)
    

    #"""决策树第五层"""
    #如果data的第一行索引名为'login',并且同时满足分裂条件0和1，那么返回'未知'。（莫名其妙的login数就变少了）
    elif index[0] == 'login' and splitTerm0 and splitTerm1:
        return '未知'+index[0]+'<'+str(globalLimit)
    
    #如果data的第一行索引名为'login',并且满足分裂条件0不满足分裂条件1，那么返回'login打点故障'。（只有login数下降了）
    elif index[0] == 'login' and splitTerm0 and not splitTerm1:
        return 'login打点故障'
    
    #如果data的第一行索引名为'login',并且不满足分裂条件0，那么data去除第一行，其他参数保持不变，进行递归
    #（如果login数没有下降，那么进行下一步判断）
    elif index[0] == 'login' and not splitTerm0:
        return decisionTree(data[1:],rawData,apLimit,forwardLimit,globalLimit)


    #"""决策树第六层"""
    #如果data的第一行索引名为'webforward',并且同时满足分裂条件0和1，那么返回'web放行故障'。（莫名其妙的webforward数就变少了）
    elif index[0] == 'webforward' and splitTerm0 and splitTerm1:
        return 'web放行故障'
    
    #如果data的第一行索引名为'webforward',并且满足分裂条件0不满足分裂条件1，那么返回'webforward打点故障'。（只有webforward数下降了）
    elif index[0] == 'webforward' and splitTerm0 and not splitTerm1:
        return 'webforward打点故障'
    
    #如果data的第一行索引名为'webforward',并且不满足分裂条件0，那么data去除第一行，其他参数保持不变，进行递归
    #（如果webforward数没有下降，那么进行下一步判断）
    elif index[0] == 'webforward' and not splitTerm0:
        return decisionTree(data[1:],rawData,apLimit,forwardLimit,globalLimit)


    #"""决策树第七层"""
    #如果data的第一行索引名为'hardforward',并且满足分裂条件0，那么返回'硬件放行故障'。（莫名其妙的hardforward数就变少了）
    elif index[0] == 'hardforward' and splitTerm0:
        return '硬件放行故障'
    
    #如果data的第一行索引名为'hardforward',并且不满足分裂条件0，那么data去除第一行，其他参数保持不变，进行递归
    #（如果hardforward数没有下降，那么进行下一步判断）
    elif index[0] == 'hardforward' and not splitTerm0:
        return decisionTree(data[1:],rawData,apLimit,forwardLimit,globalLimit)


    #"""决策树第八层"""
    #如果data的第一行索引名为'forward_rate',并且不满足分裂条件0，那么返回'正常'。（正常情况）
    elif index[0] == 'forward_rate' and not splitTerm0:
        return '正常'
    
    #如果data的第一行索引名为'forward_rate',并且满足分裂条件0，那么赋值rawData到data，globalLimit - 0.05,其他参数保持不变，进行递归
    #（如果其他参数正常，只有forward_rate变少了，那么减小分裂系数，重新进行分析，）
    elif index[0] == 'forward_rate' and not splitTerm0:
        return decisionTree(rawData,rawData,apLimit,forwardLimit,globalLimit-0.05)

In [661]:
# 定义初始参数
interested = ['forward_rate', 'dhcp', 'login','ap_count']
server = {
        'host': '121.40.211.161',
        'port': 58799,
        'user': 'hhh',
        'password': '123456',
        'db': 'hive_statistical_results',
        'charset': 'utf8mb4',
        'cursorclass': pymysql.cursors.DictCursor,
    }
table = '`funnel_model2`'
uvData, totalInfo ,apData = getRawData(server, table)
apData = apData.replace(['福建','安徽','福建省','安徽省',''],np.nan).dropna()
uvDataGood1,apDataGood = dataClean(uvData,apData)

/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:115: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [662]:
uvDay1All,uvDay2All,uvDay1Lite,uvDay2Lite = getDateData(uvDataGood1,apDataGood,1,1*7)
x = ((uvDay1Lite.reindex(uvDay2Lite.index).fillna(0)-uvDay2Lite)/uvDay2Lite)

In [668]:
troubleList = {}
for hos in x.index:
    trouble = decisionTree(x.ix[hos],x.ix[hos],-0.1,-0.4,-0.4)
    troubleList[hos] = trouble

In [669]:
t = pd.Series(troubleList)

In [670]:
troubleHos = t[t !='正常'].drop(0)

In [671]:
troubleDf = pd.DataFrame(troubleHos,columns=['故障类型'])
troubleDf.insert(0,'hos_id',troubleDf.index)

In [672]:
troubleDf

,hos_id,故障类型
57,57,重定向故障
62,62,webforward打点故障
102,102,AP离线
149,149,AP离线
158,158,web放行故障
195,195,带宽负载较高（疑似）
249,249,未知 dhcp<-0.4
298,298,网关离线
347,347,网关离线
547,547,AP离线


In [673]:
x.reindex(troubleDf.index)

,ap_count,dhcp,portal,prelogin,login,webforward,hardforward,forward_rate
57,0.000000,-0.057039,-0.529820,-0.517615,-0.502841,-0.515625,-0.530120,-0.501698
62,0.500000,-0.315789,-0.304762,-0.317073,-0.307692,-0.500000,-0.142857,0.252747
102,-0.136752,0.002454,0.014726,0.018512,0.023236,0.007106,0.058184,0.055593
149,-0.148936,0.014654,-0.021376,-0.060519,-0.098325,-0.109221,-0.073213,-0.086598
158,0.000000,-0.330976,-0.301391,-0.302548,-0.299674,-0.428571,-0.281955,0.073273
195,0.000000,0.006024,-0.239560,-0.572864,-0.572165,-0.529070,-0.978495,-0.978623
249,0.000000,-0.434411,-0.302357,-0.170765,-0.218663,-0.204687,-0.610687,-0.311668
298,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
347,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
547,-0.473684,-0.246575,-0.352273,-0.390093,-0.408027,-0.429134,-0.386973,-0.186346


In [631]:
devServer = {
        'host': 'localhost',
        'port': 3306,
        'user': 'root',
        'password': 'admin123',
        'db': 'device',
        'charset': 'utf8mb4',
        'cursorclass': pymysql.cursors.DictCursor,
    }
devTable = '`dev_info`'
devSql = "SELECT * FROM `dev_info` WHERE `date` = '2017-09-20' ORDER BY `clock` DESC LIMIT 0, 1000"

In [632]:
dev_info = sqlServer(devServer,devTable)
dev_data = dev_info.sqlGetData(devSql)
devData = dev_data.sort_values('clock',ascending=False).reset_index(drop = True)
devData[devData['clock'] == devData.ix[0,'clock']][['gwid','status']]

In [654]:
class analysis1(object):
    def __init__(self,data):
        self.data = data
    
    def 

In [655]:
r = analysis1(x)

In [656]:
r.data

,ap_count,dhcp,portal,prelogin,login,webforward,hardforward,forward_rate
hos_id,,,,,,,,
1021,-0.017857,-0.054560,-0.031697,-0.061303,-0.064378,-0.066327,-0.083721,-0.030844
572,0.000000,0.059854,0.059633,0.058480,0.054878,0.022951,0.078947,0.018015
425,0.041667,-0.124365,-0.141443,-0.210059,-0.201835,-0.210702,-0.182119,-0.065956
269,0.000000,0.033268,0.136364,0.122502,0.122654,0.289855,-0.022654,-0.054121
137,0.066667,0.004969,-0.221710,0.027542,0.034441,-0.002004,0.019035,0.013996
620,0.000000,-0.011751,-0.025373,-0.054648,-0.052882,-0.052356,0.004227,0.016168
1072,0.000000,-0.043006,-0.057616,-0.065604,-0.032715,-0.051724,-0.033251,0.010194
205,0.000000,-0.070360,-0.062324,-0.084906,-0.066729,-0.090649,-0.073810,-0.003710
1148,-0.041667,-0.027695,-0.057069,-0.074627,-0.059561,-0.090909,-0.069204,-0.042691
